In [1]:
from env.SourcingEnv import *
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import pickle as pkl
from env.HelperClasses import *
from sim.sim_functions import *
import itertools

In [2]:
filename = "output/msource_value_dic_07-04-2022-05-59-13.pkl"

with open(filename, 'rb') as f:
    output_obj = pkl.load(f)

    value_dic = output_obj["state_value_dic"]
    model_params = output_obj["model_params"]
    sourcingEnv = output_obj["mdp_env"]

    sourcingEnv = SourcingEnv(
        lambda_arrival = model_params['mdp_env_params']['lambda'], # or 10
        procurement_cost_vec = np.array(model_params['mdp_env_params']['procurement_cost_vec']),
        supplier_lead_times_vec = np.array(model_params['mdp_env_params']['supplier_lead_times_vec']),
        on_times = np.array([1, 1]), 
        off_times = np.array([np.Inf, np.Inf]))

In [3]:
state_backorders = list(itertools.product(range(MAX_INVEN - BACKORDER_MAX), range(sourcingEnv.n_suppliers)))
state_s = list(range(MAX_INVEN - BACKORDER_MAX))
state_onoff = list(itertools.product(range(2), range(sourcingEnv.n_suppliers)))

possible_state_tuples = list(itertools.product(state_s, state_backorders, state_onoff))
poss_states = [[x[0], np.array(list(x[1])), np.array(list(x[2]))] for x in possible_state_tuples]

action_space_tup = list(itertools.product(range(sourcingEnv.action_size), range(sourcingEnv.n_suppliers))) 
action_space = [np.array(list(x)) for x in action_space_tup]

m = gp.Model("MDP")
x = {}

for state in poss_states:
    for a in action_space:
        state_rep = MState(state[0], sourcingEnv.n_suppliers, state[1], state[2])
        state_rep_str = str(state_rep)
        a_rep = repr(list(a))
        cost = cost_calc_expected_di(sourcingEnv, a, custom_state = state_rep)
        x[state_rep_str, a_rep] = m.addVar(obj = cost, name='x-'+str(state)+"-"+str(a))
        m.addConstr(x[state_rep_str, a_rep] >= 0)
        

Set parameter CSQueueTimeout to value 120
Set parameter CSIdleTimeout to value 60
Set parameter ServerTimeout to value 10
Set parameter TokenServer to value "10.162.183.44"


In [5]:
# need to write a pij function
# tau = sourcingEnv.compute_event_arrival_time(a)


for j_state in poss_states:
    j_repstr = str(MState(j_state[0], sourcingEnv.n_suppliers, j_state[1], j_state[2]))
    
    poss_i_states_tuples = [] # possible prev. states

    for a_i in action_space:
        event_probs = sourcingEnv.get_event_probs(a_i)
        for k in range(sourcingEnv.n_suppliers):
            i_state_supp = j_state[1]
            i_state_supp[k] = j_state[1] - a_i[k]
            change_i_state = MState(j_state[0] + 1, sourcingEnv.n_suppliers, i_state_supp, j_state[2])
            poss_i_states_tuples.append((a_i, change_i_state, event_probs[0])) # Event DEMAND_ARRIVAL

            i_state_supp = j_state[1]
            i_state_supp[k] = j_state[1] - a_i[k] + 1
            change_i_state = MState(j_state[0] - 1, sourcingEnv.n_suppliers, i_state_supp, j_state[2])
            index = sourcingEnv.get_event_index_from_event(Event.SUPPLY_ARRIVAL, k)
            poss_i_states_tuples.append((a_i, change_i_state, event_probs[index])) # Event SUPPLY_ARRIVAL

            i_state_v = j_state[2]
            if j_state[k] == 1:
                i_state_v[k] = 0
                change_i_state = MState(j_state[0], sourcingEnv.n_suppliers, j_state[1], i_state_v)
                index = sourcingEnv.get_event_index_from_event(Event.SUPPLIER_ON, k)
                poss_i_states_tuples.append((a_i, change_i_state, event_probs[index])) # Event SUPPLY_ARRIVAL

            i_state_v = j_state[2]
            if j_state[k] == 0:
                i_state_v[k] = 1
                change_i_state = MState(j_state[0], sourcingEnv.n_suppliers, j_state[1], i_state_v)
                index = sourcingEnv.get_event_index_from_event(Event.SUPPLIER_OFF, k)
                poss_i_states_tuples.append((a_i, change_i_state, event_probs[index])) # Event SUPPLY_ARRIVAL
            
    m.addConstr(sum(x[j_repstr, repr(list(a))] for a in action_space) - sum(pij*x[str(state_i), repr(list(a_i))] for (a_i, state_i, pij) in poss_i_states_tuples) == 0)


ValueError: setting an array element with a sequence.